In [ ]:
import os
from os.path import join as oj
from collections import defaultdict

import numpy as np
import pandas as pd
from scipy.stats import sem

from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr

import sys
sys.path.append('../')
sys.path.append('../tools')
from vol_utils.utils import cwd, set_up_plotting
plt = set_up_plotting()
from exp_utils import _get_SV_estimates

from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
help(_get_SV_estimates)
methods = ['MC', 'Owen', 'Sobol', 'Stratified', 'KernelSHAP', '2-FAE-0', '2-FAE-2', '2-FAE-5', '2-FAE-100']
shorthands = ['MC','Owen', 'Sobol', 'stratified', 'kernel', 'Ours (0)', 'Ours (2)', 'Ours (5)', 'Ours (100)']
experiment_options = ["IG", "vol", "feature"]

# choose which expreriment to run
experiment_name = experiment_options[0]

In [ ]:
def save(df, name, csv=True, markdown=False, latex=True):
    if csv:
        df.to_csv(name + '.csv')

    if markdown:
        with open(name + '.md', 'w') as md_file:
            df.to_markdown(md_file, index=False)

    if latex:
        df.to_latex(name + '.latex', index=False)
    return


def get_overall_dfs(result, n_trials=5):

    stored_dict = result['res'].item()
    n = result['n']
    exact_SV = stored_dict['Exact']
    exact_SV /= np.mean(exact_SV)

    overall_dfs = []
    for t in range(n_trials):

        desirability_data  = defaultdict(list)
        mis_counts = defaultdict(float)
        mis_sums = defaultdict(float)

        afs_data = defaultdict(list)

        mape_res = defaultdict(float)
        mse_res = defaultdict(float)
        pearsonr_res = defaultdict(float)
        cosine_res = defaultdict(float)

        acc_data = defaultdict(list)

        for method, shorthand in zip(methods, shorthands):
            mcs, afs, min_afs = stored_dict[method+'statistics'][t]
            sv_estimates = _get_SV_estimates(n, mcs)

            for i in range(n):
                for j in range(n):
                    if i == j: continue
                    exact_diff = exact_SV[i] - exact_SV[j]
                    est_diff = sv_estimates[i] - sv_estimates[j]

                    mis_counts[method] += np.sign(exact_diff) != np.sign(est_diff)

                    ref_SV = max(np.abs(exact_SV[i]), np.abs(exact_SV[j]))  
                    mis_sums[method] += np.abs(exact_diff - est_diff) / ref_SV

            cols = ['inversion counts', 'inversion error' ]        
            desirability_data[shorthand] = [ mis_counts[method], mis_sums[method]]
            desirability_df = pd.DataFrame(desirability_data, index=cols)
            desirability_df.T.index.name= 'baselines'

            cols = ['(max-min)/mean', 'std', 'nlnsw']

            mcs, afs, min_afs = stored_dict[method+'statistics'][t]

            normed_afs = afs
            normed_afs = normed_afs / np.sum(normed_afs) * len(normed_afs)
            nlnsw = - np.log(normed_afs).sum()
                        
            afs_data[shorthand] = [(np.max(afs) - np.min(afs)) / np.mean(afs), np.std(afs), nlnsw]
            afs_df = pd.DataFrame(afs_data, index=cols)
            afs_df.T.index.name= 'baselines'


            cols = ['MAPE', 'MSE', 'Pearson', 'Cosine']

            mape_res[method] = mean_absolute_percentage_error(exact_SV, sv_estimates)
            mse_res[method] = mean_squared_error(exact_SV, sv_estimates)
            pearsonr_res[method], pvalue = pearsonr(exact_SV, sv_estimates)
            cosine_res[method] = cosine_similarity(exact_SV.reshape(1, -1), sv_estimates.reshape(1, -1)).squeeze()

            acc_data[shorthand] = [ mape_res[method], mse_res[method], pearsonr_res[method], cosine_res[method]]

            acc_df = pd.DataFrame(acc_data, index=cols)
            acc_df[:2].T.index.name = 'baselines'
            acc_df[:2].T

        overall_df = acc_df[:2].T.merge(desirability_df.T, on='baselines').merge(afs_df.T, on='baselines')
        overall_dfs.append(overall_df)
    return overall_dfs

def get_mean_sem_dfs(overall_dfs):

    metrics = ['MAPE', 'MSE', 'inversion counts', 'inversion error', '(max-min)/mean', 'std','nlnsw' ]

    np_arr = np.asarray([overall_df.values for overall_df in overall_dfs])
    std_errs = np.zeros((len(methods), len(metrics)))
    data_dict = {'baselines': shorthands}
    for i in range(len(methods)):
        for j in range(len(metrics)):
            std_errs[i,j] = sem(np_arr[:, i, j ])

            
    for j, metric in enumerate(metrics):
        data_dict[metric] = list(std_errs[:,j]) 
    sem_df = pd.DataFrame(data_dict, index=data_dict['baselines'])
    
    means = np.zeros((len(methods), len(metrics)))

    for i in range(len(methods)):
        for j in range(len(metrics)):
            means[i,j] = np.mean(np_arr[:, i, j ])
    data_dict = {'baselines': shorthands}

    for j, metric in enumerate(metrics):
        data_dict[metric] = list(means[:,j]) 
    mean_df = pd.DataFrame(data_dict, index=data_dict['baselines'])
    
    return mean_df, sem_df

def get_max_se_mean_ratio(mean_df, sem_df):

    m_values = mean_df.drop(['baselines'], axis=1).values
    s_values = sem_df.drop(['baselines'], axis=1).values
    max_ratio = 0
    for i in range(len(methods)):
        for j in range(6):
            se = s_values[i, j]
            avg = m_values[i, j]

            if avg != 0:
                ratio = se / avg
                if max_ratio < ratio:
                    max_ratio = ratio

    return max_ratio


def get_mean_se_str_df(mean_df, sem_df):
    m_df = mean_df.drop(['baselines'], axis=1)
    s_df = sem_df.drop(['baselines'], axis=1)

    mean_se_data_dict = {'baselines': shorthands}

    for metric in ['MAPE', 'MSE', 'inversion counts', 'inversion error', 'nlnsw', '(max-min)/mean']:
        res = []
        for avg, se in zip(m_df[metric].round(5), s_df[metric].round(5)):

            if 0.1<avg<10:
                avg_str = f'{avg:.2f}'
            else:
                avg_str = f'{avg:.2e}'
            if 0.1< se <100:
                se_str = f'{se:.2f}'
            else:
                se_str = f'{se:.1e}'

            res.append(f'{avg_str} ({se_str})' )
        mean_se_data_dict[metric] = res
    mean_se_df = pd.DataFrame(mean_se_data_dict)
    mean_se_df.set_index('baselines')
    
    return mean_se_df

In [ ]:
n_trials = 5
PATH = '../experiment_data'
results_dir = oj(PATH, experiment_name)

with cwd(results_dir):
    for result_dir in os.listdir():
        try:
            if result_dir.endswith('.npz'):
                result = np.load(result_dir, allow_pickle=True)
                overall_dfs = get_overall_dfs(result, n_trials=n_trials)
                mean_df, sem_df = get_mean_sem_dfs(overall_dfs)
                name = result_dir.replace('res_', '').replace('.npz', '')
                mean_se_str_df = get_mean_se_str_df(mean_df, sem_df)
                save(mean_se_str_df, name+'-mean-sem', csv=False)  
        except Exception as e:
            print(f'Exception is {e} with {result_dir}')